# Notebook for analyzing the results

In this notebook we analyze the results from the experiments we ran. 

In [2]:
import json
import pandas as pd
from pathlib import Path
from bnsl.scoring import compute_shd

Store the records as a dataframe

In [3]:
root = Path.cwd().parents[1]    # two levels up
files = list((root / "data" / "results").rglob("*.json"))

In [ ]:
records = []
for f in files:
    with open(f) as fp:
        r = json.load(fp)
        record = {
            "algorithm": r["algorithm"],
            "network": r["network"].split("/")[-1].split(".")[0],
            "num_samples": r["num_samples"],
            "score": r["score"],
            "theoretical_upper_bound": r["bounds"].get("theoretical_upper_bound"),
            "naive_upper_bound": r["bounds"].get("naive_upper_bound"),
            "runtime": r["seconds_elapsed"],
            "k": r["params"].get("k"),
            "l": r["params"].get("l"),
            "num_vars": r["num_variables"],
            "seed": r["seed"],
            "parent_map": r.get("parent_map"),
        }
        records.append(record)

df = pd.DataFrame(records)

In [5]:
df.head()

,algorithm,network,num_samples,score,optimal_upper_bound,runtime,k,l,num_vars,seed,parent_map
0,silander_myllymaki,survey,100,-423.143,NaN,0.000,NaN,NaN,6,42,"{'A': [], 'E': ['A'], 'O': [], 'R': [], 'S': [..."
1,silander_myllymaki,asia,10000,-22193.508,NaN,0.002,NaN,NaN,8,44,"{'asia': [], 'bronc': ['smoke'], 'dysp': ['bro..."
2,silander_myllymaki,survey,1000,-4033.283,NaN,0.000,NaN,NaN,6,44,"{'A': ['E'], 'E': [], 'O': [], 'R': [], 'S': [..."
3,silander_myllymaki,survey,10000,-39569.183,NaN,0.000,NaN,NaN,6,44,"{'A': ['E'], 'E': ['O', 'R'], 'O': ['T'], 'R':..."
4,silander_myllymaki,sachs,100,-869.895,NaN,0.017,NaN,NaN,11,42,"{'Akt': ['Mek'], 'Erk': ['Akt'], 'Jnk': ['PKA'..."


 Create a table with only approximation algorithm, and true fields from dp

In [6]:
approx_df = df[df["algorithm"] == "approximation_algorithm"].drop(columns=["algorithm"])

In [7]:
# apply the true score from silander myllymaki to the corresponding approximation results
approx_df["dp_score"] = approx_df.apply(
    lambda row: df[
        (df["algorithm"] == "silander_myllymaki") &
        (df["network"] == row["network"]) &
        (df["num_samples"] == row["num_samples"]) &
        (df["seed"] == row["seed"])
    ]["score"].values[0], axis=1
)

In [8]:
# apply the true pm from silander myllymaki to the corresponding approximation results
approx_df["dp_parent_map"] = approx_df.apply(
    lambda row: df[
        (df["algorithm"] == "silander_myllymaki") &
        (df["network"] == row["network"]) &
        (df["num_samples"] == row["num_samples"]) &
        (df["seed"] == row["seed"])
    ]["parent_map"].values[0], axis=1
)

In [9]:
approx_df.head()

,network,num_samples,score,optimal_upper_bound,runtime,k,l,num_vars,seed,parent_map,dp_score,dp_parent_map
45,sachs,100,-869.895,-684.999427,0.103,6.0,4.0,11,42,"{'Akt': ['Mek'], 'Erk': ['Akt'], 'Jnk': ['PKA'...",-869.895,"{'Akt': ['Mek'], 'Erk': ['Akt'], 'Jnk': ['PKA'..."
46,asia,1000,-2241.466,11067.515331,0.002,5.0,1.0,8,44,"{'asia': [], 'bronc': [], 'dysp': ['bronc', 'e...",-2235.024,"{'asia': [], 'bronc': ['smoke'], 'dysp': ['bro..."
47,sachs,1000,-7629.010,-3373.455302,0.018,4.0,2.0,11,42,"{'Akt': ['Erk', 'PKA'], 'Erk': ['Mek', 'PKA'],...",-7629.010,"{'Akt': ['Erk', 'PKA'], 'Erk': [], 'Jnk': ['PK..."
48,earthquake,100,-64.285,45.483534,0.001,4.0,2.0,5,44,"{'Alarm': ['JohnCalls'], 'Burglary': ['Alarm']...",-64.285,"{'Alarm': ['JohnCalls'], 'Burglary': ['Alarm']..."
49,earthquake,1000,-415.992,-68.773338,0.002,3.0,2.0,5,42,"{'Alarm': [], 'Burglary': ['Alarm'], 'Earthqua...",-415.992,"{'Alarm': [], 'Burglary': ['Alarm'], 'Earthqua..."


Compute SHD from the DP network

In [10]:
approx_df["SHD"] = approx_df.apply(
    lambda row: compute_shd(
        root / "networks" /  "small" / f"{row['network']}.bif",
       row["parent_map"],
    ), axis=1
)

In [11]:
approx_df.sort_values(["network", "num_samples"]).head(20)

,network,num_samples,score,optimal_upper_bound,runtime,k,l,num_vars,seed,parent_map,dp_score,dp_parent_map,SHD
50,asia,100,-251.306,-90.570102,0.019,6.0,4.0,8,43,"{'asia': [], 'bronc': ['smoke'], 'dysp': ['bro...",-251.306,"{'asia': [], 'bronc': ['smoke'], 'dysp': ['bro...",4
67,asia,100,-252.923,66.932818,0.003,4.0,2.0,8,43,"{'asia': [], 'bronc': [], 'dysp': ['bronc'], '...",-251.306,"{'asia': [], 'bronc': ['smoke'], 'dysp': ['bro...",4
69,asia,100,-252.923,66.932818,0.003,4.0,2.0,8,44,"{'asia': [], 'bronc': [], 'dysp': ['bronc'], '...",-251.306,"{'asia': [], 'bronc': ['smoke'], 'dysp': ['bro...",4
112,asia,100,-252.923,-92.994990,0.007,3.0,2.0,8,44,"{'asia': [], 'bronc': ['dysp'], 'dysp': [], 'e...",-251.306,"{'asia': [], 'bronc': ['smoke'], 'dysp': ['bro...",6
113,asia,100,-252.923,-92.994990,0.007,3.0,2.0,8,43,"{'asia': [], 'bronc': ['dysp'], 'dysp': [], 'e...",-251.306,"{'asia': [], 'bronc': ['smoke'], 'dysp': ['bro...",6
116,asia,100,-256.725,1007.489893,0.001,5.0,1.0,8,42,"{'asia': [], 'bronc': [], 'dysp': ['bronc'], '...",-251.306,"{'asia': [], 'bronc': ['smoke'], 'dysp': ['bro...",11
129,asia,100,-256.725,1007.489893,0.001,5.0,1.0,8,43,"{'asia': [], 'bronc': [], 'dysp': ['bronc'], '...",-251.306,"{'asia': [], 'bronc': ['smoke'], 'dysp': ['bro...",11
145,asia,100,-251.306,-90.570102,0.023,6.0,4.0,8,42,"{'asia': [], 'bronc': ['smoke'], 'dysp': ['bro...",-251.306,"{'asia': [], 'bronc': ['smoke'], 'dysp': ['bro...",4
146,asia,100,-252.923,-92.994990,0.003,3.0,2.0,8,42,"{'asia': [], 'bronc': ['dysp'], 'dysp': [], 'e...",-251.306,"{'asia': [], 'bronc': ['smoke'], 'dysp': ['bro...",6
156,asia,100,-252.923,66.932818,0.005,4.0,2.0,8,42,"{'asia': [], 'bronc': [], 'dysp': ['bronc'], '...",-251.306,"{'asia': [], 'bronc': ['smoke'], 'dysp': ['bro...",4


Make k and l ints

In [12]:
approx_df["k"] = approx_df["k"].astype(int)
approx_df["l"] = approx_df["l"].astype(int)

Plots comparing scores to upper bound

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1) Aggregate over seeds
df_agg = (
    approx_df.groupby(["num_samples", "network", "k", "l"], as_index=False)
      .agg(
          score=("score", "mean"),
          dp_score=("dp_score", "mean"),
          theoretical_upper_bound=("theoretical_upper_bound", "mean"),
            naive_upper_bound=("naive_upper_bound", "mean"),
      )
)

# 2) Loop over sample sizes + approximation ratios
for (n, k, l), sub in df_agg.groupby(["num_samples", "k", "l"]):
    
    sub = sub.sort_values("network")

    x = np.arange(len(sub))
    width = 0.25

    plt.figure(figsize=(6, 4))

    # Three bars: Approx score, DP score, UB
    plt.bar(x - width, sub["score"],       width, label="Approx score")
    plt.bar(x,         sub["dp_score"],    width, label="DP (optimal)")
    plt.bar(x + width, sub["upper_bound"], width, label="Upper bound")

    plt.xticks(x, sub["network"], rotation=30, ha="right")
    plt.ylabel("Score")
    plt.title(f"Scores per network (n={n}, k={k}, l={l})")
    plt.legend(fontsize=7)
    plt.tight_layout()

    # Save figure
    outfile = root / f"experiments/plots/plot_scores_n{n}_k{k}_l{l}.png"
    plt.savefig(outfile, dpi=200, bbox_inches="tight")
    plt.close()

    print(f"Saved: {outfile}")

Saved: /home/aurora/source/repos/bn-structure-learning/experiments/plots/plot_scores_n100_k3_l2.png
Saved: /home/aurora/source/repos/bn-structure-learning/experiments/plots/plot_scores_n100_k4_l2.png
Saved: /home/aurora/source/repos/bn-structure-learning/experiments/plots/plot_scores_n100_k5_l1.png
Saved: /home/aurora/source/repos/bn-structure-learning/experiments/plots/plot_scores_n100_k6_l4.png
Saved: /home/aurora/source/repos/bn-structure-learning/experiments/plots/plot_scores_n1000_k3_l2.png
Saved: /home/aurora/source/repos/bn-structure-learning/experiments/plots/plot_scores_n1000_k4_l2.png
Saved: /home/aurora/source/repos/bn-structure-learning/experiments/plots/plot_scores_n1000_k5_l1.png
Saved: /home/aurora/source/repos/bn-structure-learning/experiments/plots/plot_scores_n1000_k6_l4.png
Saved: /home/aurora/source/repos/bn-structure-learning/experiments/plots/plot_scores_n10000_k3_l2.png
Saved: /home/aurora/source/repos/bn-structure-learning/experiments/plots/plot_scores_n10000_k4

Plot comparing SHD

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1) Aggregate over seeds (include SHD)
df_agg = (
    approx_df.groupby(["num_samples", "network", "k", "l"], as_index=False)
             .agg(
                 score=("score", "mean"),
                 dp_score=("dp_score", "mean"),
                 upper_bound=("theoretical_upper_bound", "mean"),
                 SHD=("SHD", "mean"),
             )
)

# 2) One plot per sample size, bars = (k,l) configs, x = network, y = SHD
for n, sub in df_agg.groupby("num_samples"):
    # Pivot so each (k,l) becomes a separate series per network
    pivot = sub.pivot(index="network", columns=["k", "l"], values="SHD")

    # Ensure deterministic order
    pivot = pivot.sort_index()
    networks = pivot.index.to_list()
    configs = list(pivot.columns)  # list of (k, l) tuples

    x = np.arange(len(networks))
    num_cfgs = len(configs)
    width = 0.8 / num_cfgs  # total width ~0.8 of the tick

    plt.figure(figsize=(7, 4))

    for i, (k, l) in enumerate(configs):
        # Center the group of bars around each x
        offsets = x + (i - (num_cfgs - 1) / 2) * width
        shd_vals = pivot[(k, l)].values

        plt.bar(
            offsets,
            shd_vals,
            width,
            label=f"k={int(k)}, l={int(l)}"
        )

    plt.xticks(x, networks, rotation=30, ha="right")
    plt.ylabel("Mean SHD (over seeds)")
    plt.title(f"SHD per network for different (k, l) (n={n})")
    plt.legend(fontsize=7, title="Config")
    plt.tight_layout()

    outfile = root / f"experiments/plots/plot_shd_n{n}.png"
    plt.savefig(outfile, dpi=200, bbox_inches="tight")
    plt.close()

    print(f"Saved: {outfile}")


Saved: /home/aurora/source/repos/bn-structure-learning/experiments/plots/plot_shd_n100.png
Saved: /home/aurora/source/repos/bn-structure-learning/experiments/plots/plot_shd_n1000.png
Saved: /home/aurora/source/repos/bn-structure-learning/experiments/plots/plot_shd_n10000.png


Create one df for each network

In [16]:
# make one df per network
network_dfs = {}
for network in approx_df["network"].unique():
    network_dfs[network] = approx_df[approx_df["network"] == network].drop(columns=["network"])

# 

In [17]:
asia_df = network_dfs["asia"]

Write pandas table to latex

In [ ]:
column_order = [
    "num_samples",
    "seed",
    "l",
    "k",
    "runtime",
    "score",
    "dp_score",
    "theoretical_upper_bound",
    "SHD",
]


In [19]:
# Sort globally before grouping
approx_df = approx_df.sort_values(["seed", "num_samples", "k"])

for net, df_net in approx_df.groupby("network"):
    print(f"% --- {net} ---")

    df_print = df_net.drop(columns=["network", "parent_map", "dp_parent_map", "num_vars"])

    df_print = df_print[column_order]

    # dynamically create column format string
    colfmt = "r" * len(df_print.columns)

    # generate the tabular
    tabular = df_print.to_latex(
        index=False,
        float_format="%.3f",
        column_format=colfmt,
        escape=False,
    )

    # wrap in table + resizebox
    print(
f"""\\begin{{table}}[H]
\\centering
\\scriptsize
\\caption{{Approximation algorithm vs DP on the {net} network.}}
\\label{{tab:approx_vs_dp_{net}}}
\\resizebox{{\\textwidth}}{{!}}{{%
{tabular}
}}
\\end{{table}}

"""
    )


% --- asia ---
\begin{table}[H]
\centering
\scriptsize
\caption{Approximation algorithm vs DP on the asia network.}
\label{tab:approx_vs_dp_asia}
\resizebox{\textwidth}{!}{%
\begin{tabular}{rrrrrrrrr}
\toprule
num_samples & seed & l & k & runtime & score & dp_score & optimal_upper_bound & SHD \\
\midrule
100 & 42 & 2 & 3 & 0.003 & -252.923 & -251.306 & -92.995 & 6 \\
100 & 42 & 2 & 4 & 0.005 & -252.923 & -251.306 & 66.933 & 4 \\
100 & 42 & 1 & 5 & 0.001 & -256.725 & -251.306 & 1007.490 & 11 \\
100 & 42 & 4 & 6 & 0.023 & -251.306 & -251.306 & -90.570 & 4 \\
1000 & 42 & 2 & 3 & 0.005 & -2238.429 & -2235.024 & -573.288 & 4 \\
1000 & 42 & 2 & 4 & 0.005 & -2235.024 & -2235.024 & 1098.663 & 3 \\
1000 & 42 & 1 & 5 & 0.002 & -2241.466 & -2235.024 & 11067.515 & 10 \\
1000 & 42 & 4 & 6 & 0.021 & -2235.024 & -2235.024 & -568.181 & 1 \\
10000 & 42 & 2 & 3 & 0.007 & -22198.112 & -22193.508 & -5556.137 & 4 \\
10000 & 42 & 2 & 4 & 0.013 & -22193.508 & -22193.508 & 11095.047 & 3 \\
10000 & 42 & 1 & 5 

Generate network visualizations

In [20]:
from pgmpy.models import DiscreteBayesianNetwork

def draw_bn(pm, filename="bn.png"):
    # Convert pm into edge list
    edges = [(parent, child) for child, parents in pm.items() for parent in parents]
    bn = DiscreteBayesianNetwork(edges)

    # Convert model into pygraphviz object
    model_graphviz = bn.to_graphviz()

    # Plot the model.
    model_graphviz.draw(f"plots/{filename}", prog="dot")

In [21]:
asia = network_dfs["asia"]

In [22]:
for row_idx, row in asia.iterrows():
    k = row["k"]
    l = row["l"]
    seed = row["seed"]
    num_samples = row["num_samples"]
    filename = f"asia_k{k}_l{l}_seed{seed}_samples{num_samples}.png"
    draw_bn(row["parent_map"], filename=filename)

FileNotFoundError: [Errno 2] No such file or directory: 'plots/asia_k5_l1_seed44_samples1000.png'